# 4] Decision Trees:


    SUBJECT: Foundations of Machine Learning(CS5590)

    NAME: Aviraj Antala

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from ucimlrepo import fetch_ucirepo 

#  calculating  entropy function
def entropy(y):
    labels = np.unique(y) #let's say two diff. lable yes and no then take both unique value
    entropy_value = 0
    for label in labels:
        p = sum(y == label) / len(y) # for yes is yes then sum and divid by whole length
        entropy_value -= p * np.log2(p)
    return entropy_value


In [2]:
#  calculating Gini index
def gini(y):
    labels = np.unique(y)
    gini_value = 1
    for label in labels:
        p = sum(y == label) / len(y)
        gini_value -= p ** 2 #same as entropy just formula is different 
    return gini_value

In [3]:

# calculating information gain using entropy or Gini
def information_gain(y, X_column, threshold, criterion='entropy'): # here y is target column #x_column is feature column 
    if criterion == 'entropy':
        parent_impurity = entropy(y) #for parents we calculate entropy
    else:
        parent_impurity = gini(y) #for parents we calculate gini
    
    left_indices = X_column <= threshold # we divie them into split based on threshold let's say threshold is 5 then 5<= left, 
    right_indices = X_column > threshold #other in right
    
    if sum(left_indices) == 0 or sum(right_indices) == 0:
        return 0        #if one class is not present in the node then it is pure node so entropy become 0.
    
    n = len(y)
    n_left, n_right = sum(left_indices), sum(right_indices) #for calculating weighted probability
    
    if criterion == 'entropy':
        e_left = entropy(y[left_indices]) 
        e_right = entropy(y[right_indices])
    else:
        e_left = gini(y[left_indices])
        e_right = gini(y[right_indices])
    
    child_impurity = (n_left / n) * e_left + (n_right / n) * e_right 
    
    return parent_impurity - child_impurity #final information gain

In [4]:
# find the best split
def best_split(X, y, criterion='entropy'):
    best_gain = -1
    best_feature = None #starting best features is NONE
    best_threshold = None # same for threshold
    
    for feature_index in range(X.shape[1]): #x.shape[1] means first row which contain features
        X_column = X[:, feature_index] # all row of that particular features let's consider fixed acidity here
        thresholds = np.unique(X_column) #all unique value of fixed acidity
        
        for threshold in thresholds:
            gain = information_gain(y, X_column, threshold, criterion) #calculate IG for all unique threshold and for feature also
            if gain > best_gain:
                best_gain = gain
                best_feature = feature_index #assign best features 
                best_threshold = threshold # assign best threshold
    
    return best_feature, best_threshold


In [5]:
# Class for the decision tree
class DecisionTree:
    def __init__(self, max_depth=None, criterion='entropy'):
        self.max_depth = max_depth
        self.criterion = criterion 
        self.tree = None #intialize tree 

    def fit(self, X, y, depth=0):
        # Set label at the leaf
        if len(np.unique(y)) == 1 or (self.max_depth is not None and depth >= self.max_depth):
            return self._most_common_label(y) #this if is ude for let's say node have only one value or if depth >max_depth 
                                              #then give label to that node ex: yes or no
        feature_index, threshold = best_split(X, y, self.criterion)
        
        if feature_index is None: #if there is no any feature remain based on that we split then give label to node
            return self._most_common_label(y)
        
        left_indices = X[:, feature_index] <= threshold
        right_indices = X[:, feature_index] > threshold
        
        left_subtree = self.fit(X[left_indices], y[left_indices], depth + 1) #recursion for construct tree
        right_subtree = self.fit(X[right_indices], y[right_indices], depth + 1)
        
        return {"feature_index": feature_index, "threshold": threshold,
                "left": left_subtree, "right": right_subtree} #it will generate dictionary type node in to the trees
    
    def predict(self, X): # now let's say we have whole decision tree and particular instance x coming for test
        return np.array([self._traverse_tree(x, self.tree) for x in X]) #then we traverse to the tree
    
    def _traverse_tree(self, x, tree):
        if isinstance(tree, dict): #that particular instance is dictionary type or not
            feature_index = tree["feature_index"]
            threshold = tree["threshold"]
            if x[feature_index] <= threshold: #comparing with threshold if <= go into left subtree
                return self._traverse_tree(x, tree["left"])
            else:
                return self._traverse_tree(x, tree["right"])
        else:                       #if not that means it is leaf node where label is present
            return tree
    
    def _most_common_label(self, y): #function for assigning label
        labels, counts = np.unique(y, return_counts=True)
        return labels[np.argmax(counts)]



In [6]:
# K-fold cross-validation
def k_fold_cross_validation(X, y, clf, k=10):
    kf = KFold(n_splits=k, shuffle=True, random_state=42) #we can split it into 10 parts
    accuracies = [] #we ar making one list to count final mean acuracy 
    
    for train_index, test_index in kf.split(X):# it will take index of 9 parts 
        X_train, X_test = X[train_index], X[test_index] #assign it to x_train and one assign to X_test
        y_train, y_test = y[train_index], y[test_index] #also take target value for final vertification
        
        clf.tree = clf.fit(X_train, y_train) #creating whole tree
        predictions = clf.predict(X_test) #give data for prediction
        accuracy = np.sum(predictions == y_test) / len(y_test) #verify with original value
        accuracies.append(accuracy) # do for all 10 parts and append it to the accuracy list
    
    return np.mean(accuracies) #create final list and take mean


In [7]:
# Post-pruning function (basic implementation)
def post_prune_tree(tree, X_val, y_val, clf): #original tree and validation set pass to here
    if not isinstance(tree, dict): #if leaf node then
        return tree
    
    left_subtree = post_prune_tree(tree['left'], X_val, y_val, clf)  #goes to lastnode or left bottom node for pruning
    right_subtree = post_prune_tree(tree['right'], X_val, y_val, clf)
    
    if isinstance(left_subtree, dict) or isinstance(right_subtree, dict):
        return tree
    # here I use this method for pruning we can also do pruning based on error i comment that code also but this function give me more accuracy.
    # merge nodes if both subtrees are leaves and the same
    combined_label = clf._most_common_label(np.concatenate([y_val[X_val[:, tree['feature_index']] <= tree['threshold']],
                                                            y_val[X_val[:, tree['feature_index']] > tree['threshold']]]))
    
    if (combined_label == left_subtree and combined_label == right_subtree):
        return combined_label
    return tree





* Here I use this method for pruning we can also do pruning based on error I comment that code also but this function give me more accuracy.

* In this we can also use alpha that say that don't wait for to pure node take some error rate if error less than this 
* remove the node


In [8]:
# def post_prune_tree(tree, X_val, y_val, clf):
#     # node is a leaf node return 
#     if not isinstance(tree, dict):
#         return tree
    
#     # go till last node
#     left_subtree = post_prune_tree(tree['left'], X_val, y_val, clf)
#     right_subtree = post_prune_tree(tree['right'], X_val, y_val, clf)

#     
#     if isinstance(left_subtree, dict) or isinstance(right_subtree, dict):
#         return tree

#     check whether pruning is beneficial
#    
#     left_indices = X_val[:, tree['feature_index']] <= tree['threshold']
#     right_indices = X_val[:, tree['feature_index']] > tree['threshold']
    
#     Predictions of left and right nodes
#     left_predictions = np.full(np.sum(left_indices), left_subtree)
#     right_predictions = np.full(np.sum(right_indices), right_subtree)

#     calculate the error without pruning (ex:current node)
#     val_predictions = np.concatenate([left_predictions, right_predictions])
#     val_labels = np.concatenate([y_val[left_indices], y_val[right_indices]])
    
#     current_error = np.mean(val_predictions != val_labels) 

#     # replace with single node
#     combined_label = clf._most_common_label(val_labels)
#     combined_error = np.mean(combined_label != val_labels) 

#     # replace node with leaf
#     if combined_error <= current_error:
#         return combined_label

#     # else keep the current split
#     return tree

* Here we can enter whole data also using (red wine ,white wine) 
wine_data = fetch_ucirepo(id=186) 
* X = wine_data.data.features.to_numpy() 
* y = wine_data.data.targets  || it is mention on that website 

In [9]:
# Load dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
wine_data = pd.read_csv(url, sep=';')
X = wine_data.iloc[:, :-1].values # we take all the row of colum from 0 to upto -1
y = wine_data.iloc[:, -1].values #we take all the row of last column

# Convert to binary classification (0/1)
y = (y >= 7).astype(int) #in question it is mention that if quality >= 7 then it is 1 else 0

# now let's try to do all in one codey = (y >= 7).astype(int) #in question it is mention that if quality >= 7 then it is 1 else 0

print("Enter 1 for decision tree with entropy")
print("Enter 2 for decision tree with 10-fold cross-validation using entropy.")
print("Enter 3 for decision tree using Gini index using K-Foled cross-validation.")
print("Enter 4 for decision tree with post-pruning using K-Fold cross-validation.")



Enter 1 for decision tree with entropy
Enter 2 for decision tree with 10-fold cross-validation using entropy.
Enter 3 for decision tree using Gini index using K-Foled cross-validation.
Enter 4 for decision tree with post-pruning using K-Fold cross-validation.


In [10]:
print("Enter 1 for decision tree with entropy")
print("Enter 2 for decision tree with 10-fold cross-validation using entropy.")
print("Enter 3 for decision tree using Gini index using K-Foled cross-validation.")
print("Enter 4 for decision tree with post-pruning using K-Fold cross-validation.")

choice = int(input())
if choice == 1:
    # Case 1:for decision tree with entropy
    clf = DecisionTree(max_depth=10, criterion='entropy')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    clf.tree = clf.fit(X_train, y_train) # create decision tree based on taining
    predictions = clf.predict(X_test) #predict in testing data
    accuracy = np.sum(predictions == y_test) / len(y_test)
    print(f'Accuracy using entropy-based decision tree: {accuracy}')
    #y = (y >= 7).astype(int) #in question it is mention that if quality >= 7 then it is 1 else 0


elif choice == 2:
    # Case 2: for decision tree with 10-fold cross-validation using entropy
    clf = DecisionTree(max_depth=10, criterion='entropy')
    accuracy = k_fold_cross_validation(X, y, clf, k=10)
    print(f'Accuracy using 10-fold cross-validation with entropy: {accuracy}')

elif choice == 3:
    # Case 3: for decision tree using Gini index using K-Foled cross-validation
    clf = DecisionTree(max_depth=10, criterion='gini')
    accuracy = k_fold_cross_validation(X, y, clf, k=10)
    print(f'Accuracy with gini index decision tree with 10 k-fold: {accuracy}')

elif choice == 4:
    # Case 4: for decision tree with post-pruning using k-fold cross-validation
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    accuracies = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index] #first it doing 10 partition for k-fold (9 fold for training and 1 for testing)
        y_train, y_test = y[train_index], y[test_index]
        
        # split training set into training and validation for pruning
        split_index = int(0.8 * len(X_train)) #now here we seperate 20% of data for validation out of training and take 80% for training
        X_train_part, X_val = X_train[:split_index], X_train[split_index:] # starting index to particular split index goes into X_train_part and split_index to end goes into validation
        y_train_part, y_val = y_train[:split_index], y_train[split_index:]
        
        # train the tree and apply post-pruning
        clf = DecisionTree(max_depth=6, criterion='entropy')
        clf.tree = clf.fit(X_train_part, y_train_part) #built tree using training part
        pruned_tree = post_prune_tree(clf.tree, X_val, y_val, clf) #checking error into validation set and pass original tree
        clf.tree = pruned_tree #final pruned tree
        
        predictions = clf.predict(X_test)
        accuracy = np.sum(predictions == y_test) / len(y_test)
        accuracies.append(accuracy)
    
    average_accuracy = np.mean(accuracies)
    print(f'accuracy with post-pruning usign k-fold: {average_accuracy}')

else:
    print("invalid choice.")


Enter 1 for decision tree with entropy
Enter 2 for decision tree with 10-fold cross-validation using entropy.
Enter 3 for decision tree using Gini index using K-Foled cross-validation.
Enter 4 for decision tree with post-pruning using K-Fold cross-validation.
Accuracy with gini index decision tree with 10 k-fold: 0.889308176100629


    Answer A] Decision Tree Implementation:

Here I implement the whole decision tree using entropy function and gini function, also I take help from various references like bishop and youtube for developing intuition.




* when I press 1 it gives me accuracy approx 0.8 to 0.9.
* Accuracy is pretty high because I classify my whole data into 0 and 1 class using y = (y >= 7).astype(int)
* so if I run code without converting it into 2 classes it gives me accuracy approx 0.5 to 0.6.


    Answer B] Cross Validation:

Accuracy of using k-fold implementation:0.873687106918239

* First I use KFold method then take one accuracy list after that pass 9 parts(folds) and then verify with other 10th part(folds) <br>
* We need to that for 10 times. <br>
* Then take to mean of that accuracy list.

    Answer C] Improvement Strategies:

    1] Use Gini index instead of entropy

Accuracy of intial implementation:0.873687106918239

Accuracy after gini implementation:0.889308176100629

* Using gini index accuracy increase slightly but it doesn't show that much improvement <br>
* because whether gini gives better results or entropy depends on a particular dataset. <br>
* In this case if I don't classify data into class 0 and class 1 then entropy gives better results.(without K-fold) <br>
* but if I use k fold with gini then it increases slightly. 

    2] Prune the tree after splitting for better generalization

Accuracy of intial implementation:0.873687106918239

Accuracy after pruning :0.889308176100629

 * Post pruning technique is used for reducing overfitting in decision trees.<br>
 * but it is implemented after a full tree is grown so we can say that it does not give that much good accuracy.<br> 
 * also it require extra time for post pruning <br> 
 * But it helps to simplify the tree using removing the node that is not that valuable for the result.<br> 
 * In this particular case accuracy is not improving that much if I use it with k-fold verification. <br>  In the case of pruning accuracy also depends on the validation set.<br>  
 * It also depends on the max_depth factor if i use max_depth as 10 or 8 then also accuracy will be affected <br> 
 * So based on this we can say that for this particular case accuracy will be increased but that doesn't create that much huge difference. <br> 
 * For doing post pruning we can use multiple method like let's say if 2 node are same then combine it <br> another method is error method: here we first come to bottom leaf node calculate error for validation    set if this error is less than error on training set then we remove this node. <br> 
 * for more modification we can use alhpha also which don't wait for node be pure <br> 
 * we set particular value of alhpha and if error is less than this we can remove this <br> 
 * I tried with all three method but maximum accuracy I got into above code so I use this 